In [28]:
import numpy as np
import json
from PIL import Image
import PIL
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random
from efficientnet_pytorch import EfficientNet
import torchvision
from torchvision import transforms, datasets

model_name = 'efficientnet-b0'  # b5
image_size = EfficientNet.get_image_size(model_name)
print(image_size)
model = EfficientNet.from_pretrained(model_name, num_classes=10)
PATH = "C:/python/pill_model.pt"
device = torch.device('cpu')
model.load_state_dict(torch.load(PATH, map_location=device))

## 데이타 로드!!
batch_size  = 32
random_seed = 555
random.seed(random_seed)
torch.manual_seed(random_seed)

## make dataset
from torchvision import transforms, datasets
data_path = 'C:/pill/pill_data'
president_dataset = datasets.ImageFolder(
                                data_path,
                                transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ]))
transform = transforms.Compose([
                                    transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                ])
data_dir = 'C:/pill/test'  
img = PIL.Image.open('C:/python/test/test1.jpg')
img = data_transforms(img)

#test_data = datasets.ImageFolder(data_dir, 
#                                    transform=transform)
#testloader = torch.utils.data.DataLoader(test_data, batch_size=32)

#data_iter = iter(testloader)
#images, labels = next(data_iter)
#fig, axes = plt.subplots(1,4)
#for ii in range(4):
#    ax=axes[ii]
    
    
## data split
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
train_idx, tmp_idx = train_test_split(list(range(len(president_dataset))), test_size=0.5, random_state=random_seed)
datasets = {}
datasets['train'] = Subset(president_dataset, train_idx)
datasets['test'] = Subset(president_dataset, tmp_idx)


## data loader 선언
dataloaders, batch_num = {}, {}
dataloaders['train'] = torch.utils.data.DataLoader(datasets['train'],
                                              batch_size=batch_size, shuffle=True,
                                              num_workers=0)
dataloaders['test']  = torch.utils.data.DataLoader(datasets['test'],
                                              batch_size=batch_size, shuffle=False,
                                              num_workers=0)
batch_num['train'], batch_num['test'] = len(dataloaders['train']), len(dataloaders['test'])

criterion = nn.CrossEntropyLoss()

class_names = {
    "0": "circle",      # "0": "원형"
    "1": "hexagon",   # "1": "육각형"
    "2": "octagon",   # "2": "팔각형"
    "3": "oval",  # "3": "타원형"
    "4": "pentagon",   # "4": "오각형"
    "5": "rectangle",   # "5": "직사각형"
    "6": "rhombus",   # "6": "마름모형"
    "7": "semicircular",   # "7": "반원형"
    "8": "square",   # "8": "정사각형"
    "9": "triangle"   # "9": "삼각형"
}

def test_and_visualize_model(model, phase = 'test', num_images=2):
    model.eval()
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders[phase]):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)        
            
            # 예시 그림 plot
            for j in range(1):
                print(class_names[str(preds[j].cpu().numpy())])
                
            if i == 0 : break
    ## TEST!
test_and_visualize_model(model, phase = 'test')

224
Loaded pretrained weights for efficientnet-b0
triangle
